# Feature Engineering

## Longueur de la séquence

In [ ]:
train_enz['len_sequence']=train_enz['protein_sequence'].apply(lambda x: len(x))
test_enz['len_sequence']=test_enz['protein_sequence'].apply(lambda x: len(x))

## Liste des acides aminés

In [ ]:
AA=['P','T','W','A','C','D','L','K','H','V','R','F','Q','G','S','I','M','E','N','Y']

## Comptage des acides aminés dans chaque séquence

Inspiré du kernel 'Enzyme Stability Prediction' de MAHDEE MUSHFIQUE KAMAL mettre le lien.
[google](https://www.kaggle.com/code/mahdeemushfiquekamal/enzyme-stability-prediction)

In [ ]:
for i in AA: 
    nom_colonne = str ('nb_'+i)
    train_enz[nom_colonne] = train_enz['protein_sequence'].apply(lambda x: x.count(i))
    test_enz[nom_colonne] = test_enz['protein_sequence'].apply(lambda x: x.count(i))

In [ ]:
train_enz

In [ ]:
test_enz

# Analyse

## Observation des lettres selon des tm particulières

Observons le describe de la colonne tm afin de se rappeler qu'elles sont les valeurs minimale et maximale.

In [ ]:
train_enz['tm'].describe()

On voit ainsi que la valeur min est de 25 et la valeur max est de 111.8°c, observons maintenant qu'elle est la tm la plus présente dans notre jeu de données.

In [ ]:
print(train_enz['tm'].value_counts())

On voit ainsi, que la tm la plus présente est de 25°C, nous ferons donc un graphique afin de montrer le nombre moyen de telle ou telle lettre pour une tm de 25, et un graphique de même type pour la valeur maximale qui est de 111.8°C.

In [ ]:
tm_25=train_enz.copy()
tm_25=tm_25.loc[tm_25['tm']==25]

def createList(n):
    lst = []
    for i in range(n):
        lst.append(i)
    return(lst)

liste_index=createList(len(tm_25))

tm_25.index = liste_index

for i in AA: 
    nom_colonne = str ('nb_'+i)
    #percent_colonne=str ('%'+i)
    for j in range(0,len(tm_25)):
        tm_25.at[j,nom_colonne]=round(((tm_25.at[j,nom_colonne]/tm_25.at[j,'len_sequence'])*100),2)
tm_25

In [ ]:
tm_max=train_enz.copy()
tm_max=tm_max.loc[tm_max['tm']==111.8]

liste_index=createList(len(tm_max))

tm_max.index = liste_index

for i in AA: 
    nom_colonne = str ('nb_'+i)
    #percent_colonne=str ('%'+i)
    for j in range(0,len(tm_max)):
        tm_max.at[j,nom_colonne]=round(((tm_max.at[j,nom_colonne]/tm_max.at[j,'len_sequence'])*100),2)

tm_max

In [ ]:
letters=['nb_P','nb_T','nb_W','nb_A','nb_C','nb_D','nb_L','nb_K','nb_H','nb_V','nb_R','nb_F','nb_Q','nb_G','nb_S','nb_I','nb_M','nb_E','nb_N','nb_Y']
letters_tm=pd.DataFrame(columns=['letters','mean_25','mean_max'])
letters_tm['letters']=letters
mean_25=[]
mean_max=[]
for i in letters : 
    m=tm_25[i].mean()
    mean_25.append(m)
    o=tm_max[i].mean()
    mean_max.append(o)
letters_tm['mean_25']=mean_25
letters_tm['mean_max']=mean_max
letters_tm

In [ ]:
letters = letters_tm['letters']
tm_25 = letters_tm['mean_25']
tm_111 = letters_tm['mean_max']
X_axis = np.arange(len(letters))
  
plt.bar(X_axis - 0.2, tm_25, 0.4, label = 'tm_25')
plt.bar(X_axis + 0.2, tm_111, 0.4, label = 'tm_111')
  
plt.xticks(X_axis, letters)
plt.xlabel("Acide Aminés")
plt.ylabel("Nombre d'acide aminés dans la séquence")
plt.title("Nombre d'acide aminés dans la séquence selon la tm")
plt.legend()
plt.show()

On voit ainsi, que pour certains acide aminés, on a tendance, à voir une augmentation du pourcentage de la présence de ces derniers, pour les séquences avec une tm importante. Notamment l’Acide Aspartique (D), la Proline (P), la Cystéine (C) ou encore la Gmycine (G). Cependant, à l’inverse, certains acides aminés disparaissent, telle que l’Histidine (H), l’Alanine (A), l’Arginine (R) et la Sérine (S). Certains acides aminés jouent alors peut être un rôle dans les résultats que nous obtiendront. Mais avant cela, observons les corrélations entre nos variables numériques.

## Corrélation entre nos features quantitatives

In [ ]:
plt.rcParams['figure.figsize']=(30,15)
sns.heatmap(train_enz.corr(), vmin = -1, vmax = +1, cmap='coolwarm', annot=True)

On voit ici, que les features liées à la longeur de la séquence et aux nombres des différents acides aminés, sont très corrélées entre elles. Les corrélations avec les différentes features et pH/tm sont plus faible, En effet, pour la tm nous avons une corrélation maximale de -0.13, obtenu avec les acides aminés Asparagine (N), Glutamine (Q) et Cystéine (C). Pour le Ph nous avons une corrélation maximale de 0.11 avec l’Histidine (H). Nous pouvons dire ici au vu des corrélations entre la tm et les autres features que nous éviterons le dataleakage.